In [9]:
import numpy as np 
import math 


In [17]:
def calculate_tail_effects(N_sms, M,N, block_M, block_N, n_blocks_per_SM):
    n_blocks_per_grid = math.ceil(M/block_M)*math.ceil(N/block_N)
    assert n_blocks_per_grid % n_blocks_per_SM == 0, "cant launch factional blocks on a streaming multiprocessor"
    n_sm_chunks = n_blocks_per_grid // n_blocks_per_SM 
    n_full_occ_epochs = n_sm_chunks // N_sms 
    n_tail_sm_chunks = n_sm_chunks % N_sms 
    return n_full_occ_epochs, n_tail_sm_chunks, N_sms - n_tail_sm_chunks



def tail_effects(time_per_block, K, block_M, block_N, n_blocks_per_SM, n_full_epochs, n_tail_sm_chunks, left_over_sms, N_sms): 
    flops_per_block = block_M*block_N*2*K 
    time_per_sm = time_per_block
    flops_per_sm = flops_per_block*n_blocks_per_SM
    time_per_full_occupancy_epoch = time_per_sm 
    flops_per_full_occupancy_epoch = flops_per_sm*N_sms
    time_per_n_full_epochs = time_per_full_occupancy_epoch*n_full_epochs
    flops_per_n_full_epochs = flops_per_full_occupancy_epoch*n_full_epochs
    flops_n_full_epochs_p_tail = (flops_per_sm)*n_tail_sm_chunks + (flops_per_n_full_epochs)
    time_n_full_epochs_p_tail = time_per_sm + time_per_n_full_epochs 
    low_bound_flop_per_sec = flops_per_n_full_epochs/time_per_n_full_epochs
    upper_bound_flop_per_sec = ((n_full_epochs+1)*flops_per_full_occupancy_epoch)/(time_n_full_epochs_p_tail)
    true_flop_per_sec = (flops_n_full_epochs_p_tail)/time_n_full_epochs_p_tail

    return low_bound_flop_per_sec/1e12, upper_bound_flop_per_sec/1e12, true_flop_per_sec/1e12

In [18]:
N_sms = 188
M = 4096
N = 4096 
block_M = 128
block_N = 128 
n_blocks_per_SM = 2 

n_full_epochs, n_tail_sm_chunks, left_over_sms = calculate_tail_effects(N_sms, M, N, block_M, block_N, n_blocks_per_SM)

In [19]:
print(n_full_epochs, n_tail_sm_chunks, left_over_sms)

2 136 52


In [ ]:
lower,upper_true = tail_effects(0.2e-3,4096,block_M, block_N, n_blocks_per_SM, n_full_epochs, n_tail_sm_chunks, left_over_sms, N_sms)

In [23]:
print(upper,true,lower)

252.32932864 252.32932863999997 229.0649224533333
